# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
import requests
import ollama

from IPython.display import Markdown, display, update_display
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

In [ ]:
system_prompt = """
You are an assistant designed to analyze the user's technical question carefully and provide a well-structured, detailed, and clearly explained answer. 

Your response should include:
- A concise introduction to the problem.
- A step-by-step explanation or solution.
- Relevant code examples (if applicable).
- Clear and logical reasoning.
- Use Markdown formatting for proper structure, including headers, code blocks, lists, and emphasis (bold, italics) as needed to enhance readability.

Always ensure your answer is clear, accurate, and easy to follow for someone with basic to intermediate technical knowledge.
"""

user_question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

def get_user_prompt(question):
    user_prompt = f"Please provide a detailed, well-structured, and clear explanation to the following question: {question}"
    return user_prompt

In [ ]:
get_user_prompt(user_question)

In [ ]:
# Get gpt-4o-mini to answer, with streaming
def stream_question(user_question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(user_question)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_question(user_question)

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
def stream_llama_question(user_question):

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_user_prompt(user_question)}
    ]
    
    # Get Llama 3.2 to answer
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=messages,
        stream=True
    )
    
    display_handle = display(Markdown(""), display_id=True)
    response_text = ""
    
    for chunk in response:
        response_text += chunk['message']['content'] or ''
        response_text = response_text.replace("```","").replace("markdown", "")
        update_display(Markdown(response_text), display_id=display_handle.display_id)

In [ ]:
stream_llama_question(user_question)